In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import h5py
from selenium import webdriver
import time

In [2]:
base_dir="https://magic.wizards.com/"
base_url="https://magic.wizards.com/en/content/deck-lists-magic-online-products-game-info"

In [3]:
# create a new chrome page. Needs chromedriver.exe to be in path. I put next to anaconda stuff
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(base_url)
driver.maximize_window()

In [4]:
#Uses filter to access only certain types of decks

driver.execute_script("window.scrollTo(0, 200)") 
Filterby=driver.find_element_by_id('event_filter')
Filterby.click()
time.sleep(1)
Modern_button=driver.find_element_by_xpath("//li[contains(text(), 'Modern')]")
Modern_button.click()
time.sleep(1)
Search_button=driver.find_element_by_id('custom-search-submit')
Search_button.click()

# select by visible text
# select.select_by_visible_text('Modern')

In [5]:
#After opening the url above, Selenium clicks the scrolls through all the see more archives
'''
scrolls to bottom of page
clicks da fish link to see more info
compares old window height with new one
if equal, continually checks for 60 seconds
if not continues to scroll to the bottom and clicks the link to see more
timeout= time willing to wait for page to load for each click of the see more button
'''

SCROLL_PAUSE_TIME = 1

timeout = 1

while True:

    # Get scroll height
    ### This is the difference. Moving this *inside* the loop
    ### means that it checks if scrollTo is still scrolling 
    last_height = driver.execute_script("return document.body.scrollHeight")

    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    # touch da fishy
    da_fishy = driver.find_element_by_class_name('see-more-article-listing-section') #See more
    da_fishy.click() #click see more

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    i=0
    while (new_height == last_height) and (i<timeout) :

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        time.sleep(1)
        i+=1
    if i==timeout:
        break



In [6]:
#Handoff to BS and find the aricles we want
soup=BeautifulSoup(driver.page_source,"html.parser")
# Opens all links and gets all data
links=[]
for link in soup.find("div",{"class":"articles-listing"}).find_all('a'):
    a=link.get('href')
    links.append(a)

In [14]:
# Meat of the scrape
'''
Go to each link and scrape the useful info

'''
i=1
j=1
Headers=['Deck Name', 'Card Type','Card Count', 'Card Name', 'Card Link', \
         'Sideboard Card Count', 'Sideboard Card Name', 'Sideboard Link']
Data_set=pd.DataFrame(columns=Headers)
deck_data_ls=[]
deck_name_ls=[]
for page in links:
    page_url=base_dir+page
    r=requests.get(page_url)
    c=r.content
    page_soup=BeautifulSoup(c,"html.parser")
#     page_list.append(page_soup)
    j+=1
    for deckname in page_soup.find_all("div",{"class":"deck-group"}):
        deck_name=deckname.find("span",{"class":"deck-meta"}).h4.string
        deck_name_ls.append(deck_name)
        card_type=[]
        card_count=[]
        card_name=[]
        card_link=[]
#         sideb_type=[]
        sideb_count=[]
        sideb_name=[]
        sideb_link=[]
        for card in deckname.find("div", {"class":"sorted-by-overview-container"}).find_all("div",{"class":"element"}):
            card_type.append(card.h5.text)
            card_count.append(card.find("span",{"class":"card-count"}).text)
            card_name.append(card.find("span",{"class":"card-name"}).text)
            card_link.append(card.find("span",{"class":"card-name"}).a.get('href'))
#             sideb_type.append(np.chararray.encode(card.find("div", {"class":"sorted-by-sideboard-container"}).h5.text, 'utf8'))
        for sideb in deckname.find("div", {"class":"sorted-by-sideboard-container"}).find_all("span",{"class":"row"}):
            sideb_count.append(sideb.find("span",{"class":"card-count"}).text)
            sideb_name.append(sideb.find("span",{"class":"card-name"}).text)
            try:
                sideb_link.append(sideb.find("span",{"class":"card-name"}).a.get('href'))
            except AttributeError:
                sideb_link.append('NaN')
        deck_data=dict(zip(Headers,[deck_name,card_type, card_count,\
                             card_name, card_link, sideb_count, sideb_name, sideb_link])) 
        deck_data_ls.append(deck_data)
    
    Data_set.append(deck_data,ignore_index=True)

                
#         deck=deck_name.require_group('Deck')
#         ctype=deck.require_group('Card Types')
#         ccount=deck.require_group('Card Counts')
#         cname=deck.require_group('Card Names')
#         clink=deck.require_group('Card Links')
#         sideb=deck_name.require_group('Sideboard')
#         sideb_ctype=sideb.require_group('Sideboard Card Types')
#         sideb_ccount=sideb.require_group('Sideboard Card Counts')
#         sideb_cname=sideb.require_group('Sideboard Card Names')
#         sideb_clink=sideb.require_group('Sideboard Card Links')
        
#         ctype.create_dataset('Card Type',(len(card_type),1), str,card_type)
#         ccount.create_dataset('Card Count',(len(card_count),1),str,card_count)
#         cname.create_dataset('Card Name',(len(card_name),1),str,card_name)
#         clink.create_dataset('Card Link',(len(card_link),1),str,card_link)

#         sideb_ctype.create_dataset('Card Type',(len(sideb_type),1),'S13',card_type)
#         sideb_ccount.create_dataset('Card Count',(len(sideb_count),1),'S13',card_count)
#         sideb_cname.create_dataset('Card Name',(len(sideb_name),1),'S13',card_name)
#         sideb_clink.create_dataset('Card Link',(len(sideb_link),1),'S13',card_link)
                                 
                                 
#         for a in range(len(temp_deck)):
#             decks_list.append(temp_deck[a])
#         for deck in decks.find("div",{"class":"deck-list-text"}):
#             CreatureandNumber.append(deck.find('h5'))
#         for deck_attr in page_soup.find("div",{"class":"deck-list-text"}):
#             for card_attr in page_soup.find("div", {"class":"sorted-by-overview-container"}):

In [21]:
card_type_ls_string

['Creature (9)',
 'Sorcery (5)',
 'Instant (24)',
 'Land (22)',
 'Planeswalker (3)',
 'Creature (8)',
 'Sorcery (3)',
 'Instant (20)',
 'Land (26)',
 'Creature (31)',
 'Instant (7)',
 'Land (22)',
 'Creature (10)',
 'Sorcery (5)',
 'Instant (23)',
 'Land (22)',
 'Planeswalker (4)',
 'Creature (14)',
 'Sorcery (9)',
 'Instant (10)',
 'Land (23)',
 'Creature (11)',
 'Sorcery (7)',
 'Instant (15)',
 'Enchantment (4)',
 'Land (23)',
 'Creature (23)',
 'Sorcery (4)',
 'Instant (7)',
 'Artifact (2)',
 'Land (24)',
 'Planeswalker (5)',
 'Creature (14)',
 'Sorcery (7)',
 'Instant (10)',
 'Land (24)',
 'Creature (26)',
 'Sorcery (4)',
 'Instant (7)',
 'Land (23)',
 'Creature (29)',
 'Instant (8)',
 'Land (23)',
 'Planeswalker (3)',
 'Creature (10)',
 'Sorcery (12)',
 'Instant (11)',
 'Land (25)',
 'Planeswalker (3)',
 'Creature (15)',
 'Sorcery (11)',
 'Instant (7)',
 'Land (24)',
 'Creature (17)',
 'Sorcery (8)',
 'Instant (15)',
 'Land (20)',
 'Creature (29)',
 'Instant (4)',
 'Artifact (4)',

In [ ]:
decks=[]
for i in range(len(pages)):
    decks.append(pages[i].find("div",{"class":"decklists"}))

    

In [ ]:
deck=[]
for j in range(len(decks)):
        deck.append(decks[j].find("div",{"class":"deck-list-text"}))

In [ ]:
deck[0].find_all('h5')[0].text

In [ ]:
card_type_ls_string

In [ ]:
(len(card_type),1)

In [ ]:
encode(card_link,"ascii","ignore")

In [ ]:
card_type_ls_string[0]

In [ ]:
test=h5py.File('test.hdf5')

In [ ]:
testsub=test.create_group('test2')

In [ ]:
a='hi'
b=['how', 'are', 8]
c=['i',7]
names=['Title','Card Name',\
       'Card Type']
d=dict(zip(names,[a,b,c]))

In [ ]:
df=pd.DataFrame(columns=names)

In [ ]:
df.append(d, ignore_index=True)

In [17]:
Data_set.shape

(0, 9)

In [18]:
deck_data_ls

[{'Deck Name': 'ninell (4-0)',
  'Card Type': [array(b'Creature (9)', dtype='|S12'),
   array(b'Sorcery (5)', dtype='|S11'),
   array(b'Instant (24)', dtype='|S12'),
   array(b'Land (22)', dtype='|S9')],
  'Card Count': ['2', '1', '3', '1'],
  'Card Name': ['Gurmag Angler',
   'Rise // Fall',
   'Cryptic Command',
   'Blood Crypt'],
  'Card Link': ['http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BGurmag%5D+%5BAngler%5D',
   'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BRise%5D+%5B&%5D+%5BFall%5D',
   'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BCryptic%5D+%5BCommand%5D',
   'http://gatherer.wizards.com/Pages/Search/Default.aspx?name=+%5BBlood%5D+%5BCrypt%5D'],
  'Sideboard Card Type': [],
  'Sideboard Card Count': ['2',
   '1',
   '2',
   '1',
   '1',
   '2',
   '1',
   '1',
   '1',
   '1',
   '2'],
  'Sideboard Card Name': ['Anger of the Gods',
   'Damnation',
   'Dispel',
   'Engineered Explosives',
   'Flashfreeze',
   'Fulminator 

In [19]:
Data_set.append(deck_data_ls)

,Deck Name,Card Type,Card Count,Card Name,Card Link,Sideboard Card Type,Sideboard Card Count,Sideboard Card Name,Sideboard Link
0,ninell (4-0),"[b'Creature (9)', b'Sorcery (5)', b'Instant (2...","[2, 1, 3, 1]","[Gurmag Angler, Rise // Fall, Cryptic Command,...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2]","[Anger of the Gods, Damnation, Dispel, Enginee...",[http://gatherer.wizards.com/Pages/Search/Defa...
1,Ark4n (4-0),"[b'Planeswalker (3)', b'Creature (8)', b'Sorce...","[1, 1, 3, 2, 4]","[Elspeth, Sun's Champion, Glen Elendra Archmag...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[1, 1, 2, 2, 4, 2, 3]","[Negate, Supreme Verdict, Celestial Purge, Dis...",[http://gatherer.wizards.com/Pages/Search/Defa...
2,TonyMontana (3-1),"[b'Creature (31)', b'Instant (7)', b'Land (22)']","[2, 3, 1]","[Anafenza, Kin-Tree Spirit, Chord of Calling, ...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 1, 1, 1, 3, 1, 1, 1, 1, 3]","[Abrupt Decay, Burrenton Forge-Tender, Eidolon...",[http://gatherer.wizards.com/Pages/Search/Defa...
3,Konsti (3-1),"[b'Creature (10)', b'Sorcery (5)', b'Instant (...","[1, 1, 2, 1]","[Grim Lavamancer, Languish, Cryptic Command, B...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]","[Dispel, Anger of the Gods, Countersquall, Dur...",[http://gatherer.wizards.com/Pages/Search/Defa...
4,FakiVll (3-1),"[b'Planeswalker (4)', b'Creature (14)', b'Sorc...","[4, 4, 1, 2, 4]","[Liliana of the Veil, Dark Confidant, Duress, ...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 2, 2, 4, 2, 3]","[Terminate, Thoughtseize, Drown in Sorrow, Ful...",[http://gatherer.wizards.com/Pages/Search/Defa...
5,Navas (3-1),"[b'Creature (11)', b'Sorcery (7)', b'Instant (...","[4, 1, 2, 4, 1]","[Deceiver Exarch, Flame Slash, Cryptic Command...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1]","[Dispel, Roast, Ancient Grudge, Anger of the G...",[http://gatherer.wizards.com/Pages/Search/Defa...
6,gaku0221 (3-1),"[b'Creature (23)', b'Sorcery (4)', b'Instant (...","[1, 4, 4, 2, 2]","[Burning-Tree Shaman, Search for Tomorrow, Sum...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[1, 2, 1, 2, 2, 4, 1, 2]","[Anger of the Gods, Engineered Explosives, Hel...",[http://gatherer.wizards.com/Pages/Search/Defa...
7,NecNick (3-1),"[b'Planeswalker (5)', b'Creature (14)', b'Sorc...","[1, 4, 4, 2, 4]","[Chandra, Pyromaster, Dark Confidant, Inquisit...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 2, 1, 1, 3, 1, 1, 1, 2, 1]","[Ancient Grudge, Anger of the Gods, Boil, Dure...",[http://gatherer.wizards.com/Pages/Search/Defa...
8,Teletender (3-1),"[b'Creature (26)', b'Sorcery (4)', b'Instant (...","[3, 4, 3, 2]","[Birds of Paradise, Lingering Souls, Abrupt De...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 4, 4, 2, 3]","[Choke, Fulminator Mage, Kor Firewalker, Rest ...",[http://gatherer.wizards.com/Pages/Search/Defa...
9,Kerrick_ (3-1),"[b'Creature (29)', b'Instant (8)', b'Land (23)']","[4, 4, 1]","[Birds of Paradise, Chord of Calling, Fire-Lit...",[http://gatherer.wizards.com/Pages/Search/Defa...,[],"[2, 2, 1, 3, 1, 1, 1, 1, 3]","[Fulminator Mage, Avalanche Riders, Eidolon of...",[http://gatherer.wizards.com/Pages/Search/Defa...
